In [2]:
import glob
import csv
import pickle

In [9]:
## define a function to translate the string data into a float list

def str2array(string):
    
    together = ''
    for i in string:
        if(i != '['):
            together += i
    
    first = together.split("],")
    ret = []
    
    for i in range(len(first)):
        if(i==len(first)-1):
            second = ''
            for j in first[i]:
                if(j!=']'):
                    second += j
            second = second.split(",")
        else:
            second = first[i].split(",")
        ret.append([float(second[0]), float(second[1])])
    return ret

In [12]:
## read in the data and seperate it into two parts
## (one set is the fully recorded data, the other one is the set of not complete data)
files = glob.glob("./split_data/part*.csv")
data_false = []
data_true = []
for table in files:
    with open(table,'r') as file:
        lines = list(csv.reader(file))
        for i in range(1, len(lines)):
            if(len(lines[i])==0):
                continue
            if(lines[i][7]=="False"):
                if(len(lines[i][8])==2):
                    data_false.append([])
                else:
                    track = str2array(lines[i][8])
                    data_false.append(track)
            elif(lines[i][7]=="True"):
                if(len(lines[i][8])==2):
                    data_true.append([])
                else:
                    track = str2array(lines[i][8])
                    data_true.append(track)

In [13]:
## save the data into a pickle file
data_dict = {'data_true': data_true, 'data_false': data_false}
with open('./trackdata.pickle','wb') as file:
    pickle.dump(data_dict, file)